In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
import argparse
import os
import os.path as osp
import shutil
import numpy as np
import pickle
import yaml
from copy import deepcopy
from itertools import repeat
from torch_geometric.data import DataLoader, InMemoryDataset, download_url, extract_zip
from torch_geometric.io import read_tu_data
from torch_geometric.utils import degree
from torch.nn import Sequential, Linear, ReLU
from torch_geometric.nn import GINConv, global_add_pool, global_mean_pool, global_max_pool
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from ogb.graphproppred import PygGraphPropPredDataset
from torch_geometric.loader import DataLoader
import sys
sys.path.append('../utils')
from dataset import TUDataset_aug, OGBDataset_aug
from utils import setup_seed, evaluate_embedding

In [ ]:
# Data
name_list = ["hiv", "bbbp", "clintox", "tox21", "sider"]
i = 0
dataset = PygGraphPropPredDataset(name = "ogbg-mol" + name_list[i], root = '../data/')
 
split_idx = dataset.get_idx_split() 
train_loader = DataLoader(dataset[split_idx["train"]], batch_size=32, shuffle=True)
valid_loader = DataLoader(dataset[split_idx["valid"]], batch_size=32, shuffle=False)
test_loader = DataLoader(dataset[split_idx["test"]], batch_size=32, shuffle=False)

In [ ]:
class DisenEncoder(torch.nn.Module):
    def __init__(self, num_features, emb_dim, num_layer, K, head_layers, if_proj_head=False, drop_ratio=0.0,
                 graph_pooling='add', JK='last', residual=False, device=None, args=None):
        super(DisenEncoder, self).__init__()
        self.args = args
        self.device = device
        self.num_features = num_features
        self.K = K
        self.d = emb_dim // self.K
        self.num_layer = num_layer
        self.head_layers = head_layers
        self.gc_layers = self.num_layer - self.head_layers
        self.if_proj_head = if_proj_head
        self.drop_ratio = drop_ratio
        self.graph_pooling = graph_pooling
        if self.graph_pooling == "sum" or self.graph_pooling == 'add':
            self.pool = global_add_pool
        elif self.graph_pooling == "mean":
            self.pool = global_mean_pool
        elif self.graph_pooling == "max":
            self.pool = global_max_pool
        else:
            raise ValueError("Invalid graph pooling type.")
        self.JK = JK
        if JK == 'last':
            pass
        elif JK == 'sum':
            self.JK_proj = Linear(self.gc_layers * emb_dim, emb_dim)
        else:
            assert False
        self.residual = residual
        self.convs = torch.nn.ModuleList()
        self.bns = torch.nn.ModuleList()
        self.disen_convs = torch.nn.ModuleList()
        self.disen_bns = torch.nn.ModuleList()

        for i in range(self.gc_layers):
            if i == 0:
                nn = Sequential(Linear(num_features, emb_dim), ReLU(), Linear(emb_dim, emb_dim))
            else:
                nn = Sequential(Linear(emb_dim, emb_dim), ReLU(), Linear(emb_dim, emb_dim))
            conv = GINConv(nn)
            bn = torch.nn.BatchNorm1d(emb_dim)

            self.convs.append(conv)
            self.bns.append(bn)

        for i in range(self.K):
            for j in range(self.head_layers):
                if j == 0:
                    nn = Sequential(Linear(emb_dim, self.d), ReLU(), Linear(self.d, self.d))
                else:
                    nn = Sequential(Linear(self.d, self.d), ReLU(), Linear(self.d, self.d))
                conv = GINConv(nn)
                bn = torch.nn.BatchNorm1d(self.d)

                self.disen_convs.append(conv)
                self.disen_bns.append(bn)

        self.proj_heads = torch.nn.ModuleList()
        for i in range(self.K):
            nn = Sequential(Linear(self.d, self.d), ReLU(inplace=True), Linear(self.d, self.d))
            self.proj_heads.append(nn)

    def _normal_conv(self, x, edge_index, batch):
        xs = []
        for i in range(self.gc_layers):
            x = self.convs[i](x, edge_index)
            x = self.bns[i](x)
            if i == self.gc_layers - 1:
                x = F.dropout(x, self.drop_ratio, training=self.training)
            else:
                x = F.dropout(F.relu(x), self.drop_ratio, training=self.training)

            if self.residual and i > 0:
                x += xs[i - 1]
            xs.append(x)
        if self.JK == 'last':
            return xs[-1]
        elif self.JK == 'sum':
            return self.JK_proj(torch.cat(xs, dim=-1))

    def _disen_conv(self, x, edge_index, batch):
        x_proj_list = []
        x_proj_pool_list = []
        for i in range(self.K):
            x_proj = x
            for j in range(self.head_layers):
                tmp_index = i * self.head_layers + j
                x_proj = self.disen_convs[tmp_index](x_proj, edge_index)
                x_proj = self.disen_bns[tmp_index](x_proj)
                if j != self.head_layers - 1:
                    x_proj = F.relu(x_proj)
            x_proj_list.append(x_proj)
            x_proj_pool_list.append(self.pool(x_proj, batch))
        if self.if_proj_head:
            x_proj_pool_list = self._proj_head(x_proj_pool_list)
        x_graph_multi = torch.stack(x_proj_pool_list)
        x_node_multi = torch.stack(x_proj_list)
        x_graph_multi = x_graph_multi.permute(1, 0, 2).contiguous()
        x_node_multi = x_node_multi.permute(1, 0, 2).contiguous()
        return x_graph_multi, x_node_multi

    def _proj_head(self, x_proj_pool_list):
        ret = []
        for k in range(self.K):
            x_graph_proj = self.proj_heads[k](x_proj_pool_list[k])
            ret.append(x_graph_proj)
        return ret

    def forward(self, x, edge_index, batch, device="mps"):
        if x is None:
            x = torch.ones((batch.shape[0], 1)).to(device)
        h_node = self._normal_conv(x, edge_index, batch)
        h_graph_multi, h_node_multi = self._disen_conv(h_node, edge_index, batch)
        return h_graph_multi, h_node_multi

    def get_embeddings(self, loader):
        device = self.device
        ret = []
        y = []
        with torch.no_grad():
            for data in loader:
                data = data[0]
                data.to(device)
                x, edge_index, batch = data.x, data.edge_index, data.batch
                if x is None:
                    x = torch.ones((batch.shape[0], 1)).to(device)
                x, _ = self.forward(x, edge_index, batch)
                B, K, d = x.size()
                x = x.view(B, K * d)
                ret.append(x.cpu().numpy())
                y.append(data.y.cpu().numpy())
        ret = np.concatenate(ret, 0)
        y = np.concatenate(y, 0)
        return ret, y

class DGCL(nn.Module):
    def __init__(self, num_features, hidden_dim, num_layer, device, args):
        super(DGCL, self).__init__()

        self.args = args
        self.device = device
        self.T = self.args.tau
        self.K = args.num_latent_factors
        self.embedding_dim = hidden_dim
        self.d = self.embedding_dim // self.K

        self.center_v = torch.rand((self.K, self.d), requires_grad=True).to(device)

        self.encoder = DisenEncoder(
            num_features=num_features,
            emb_dim=hidden_dim,
            num_layer=num_layer,
            K=args.num_latent_factors,
            head_layers=args.head_layers,
            device=device,
            args=args,
            if_proj_head=args.proj > 0,
            drop_ratio=args.drop_ratio,
            graph_pooling=args.pool,
            JK=args.JK,
            residual=args.residual > 0
        )

        self.init_emb()

    def init_emb(self):
        initrange = -1.5 / self.embedding_dim
        for m in self.modules():
            if isinstance(m, nn.Linear):
                torch.nn.init.xavier_uniform_(m.weight.data)
                if m.bias is not None:
                    m.bias.data.fill_(0.0)

    def forward(self, x, edge_index, batch, num_graphs):
        if x is None:
            x = torch.ones(batch.shape[0]).to(device)
        z_graph, _ = self.encoder(x, edge_index, batch)
        return z_graph

    def loss_cal(self, x, x_aug):
        T = self.T
        T_c = 0.2
        B, H, d = x.size()
        ck = F.normalize(self.center_v)
        p_k_x_ = torch.einsum('bkd,kd->bk', F.normalize(x, dim=-1), ck)
        p_k_x = F.softmax(p_k_x_ / T_c, dim=-1)
        x_abs = x.norm(dim=-1)
        x_aug_abs = x_aug.norm(dim=-1)
        x = torch.reshape(x, (B * H, d))
        x_aug = torch.reshape(x_aug, (B * H, d))
        x_abs = torch.squeeze(torch.reshape(x_abs, (B * H, 1)), 1)
        x_aug_abs = torch.squeeze(torch.reshape(x_aug_abs, (B * H, 1)), 1)
        sim_matrix = torch.einsum('ik,jk->ij', x, x_aug) / (1e-8 + torch.einsum('i,j->ij', x_abs, x_aug_abs))
        sim_matrix = torch.exp(sim_matrix / T)
        pos_sim = sim_matrix[range(B * H), range(B * H)]
        score = pos_sim / (sim_matrix.sum(dim=-1) - pos_sim)
        p_y_xk = score.view(B, H)
        q_k = torch.einsum('bk,bk->bk', p_k_x, p_y_xk)
        q_k = F.normalize(q_k, dim=-1)
        elbo = q_k * (torch.log(p_k_x) + torch.log(p_y_xk) - torch.log(q_k))
        loss = - elbo.view(-1).mean()
        return loss

In [ ]:
def arg_parse(DS="MUTAG"):
    parser = argparse.ArgumentParser(description='DGCL Arguments.')

    parser.add_argument('--DS', type=str, default=DS)
    parser.add_argument('--lr', type=float)
    parser.add_argument('--num_gc_layers', type=int)
    parser.add_argument('--hidden_dim', type=int)
    parser.add_argument('--epoch', type=int)
    parser.add_argument('--batch', type=int)
    parser.add_argument('--aug', type=str)
    parser.add_argument('--tau', type=float)
    parser.add_argument('--drop_ratio', type=float)
    parser.add_argument('--num_latent_factors', type=int)
    parser.add_argument('--head_layers', type=int)
    parser.add_argument('--JK', type=str, choices=['last', 'sum'])
    parser.add_argument('--residual', type=int, choices=[0, 1])
    parser.add_argument('--proj', type=int, choices=[0, 1])
    parser.add_argument('--pool', type=str, choices=['mean', 'sum', 'max'])
    parser.add_argument('--fe', type=int)
    parser.add_argument('--seed', type=int)
    parser.add_argument('--num_workers', type=int)
    parser.add_argument('--log_dir', type=str)
    parser.add_argument('--log_interval', type=int, default=5)
    parser.add_argument("--debug", action='store_true', default=False)

    args, unknown = parser.parse_known_args([])

    with open(f'config/{DS}.yml', 'r') as f:
        config_yaml = yaml.load(f, Loader=yaml.FullLoader)
    for k, v in vars(args).items():
        if v is not None:
            config_yaml[k] = v
    config_ns = argparse.Namespace(**config_yaml)

    return config_ns

In [ ]:
args = arg_parse()
setup_seed(args.seed)

accuracies = {'result': [], 'result_val': []}
epochs = args.epoch
log_interval = args.log_interval
batch_size = args.batch
lr = args.lr
DS = args.DS
args.hidden_dim = int(args.hidden_dim // args.num_latent_factors) * args.num_latent_factors
path = './data'
dataset = TUDataset_aug(root=path, name=DS, aug=args.aug).shuffle()
dataset_eval = TUDataset_aug(root=path, name=DS, aug='none').shuffle()
print(len(dataset))
print(dataset.get_num_feature())
try:
    dataset_num_features = dataset.get_num_feature()
except:
    dataset_num_features = 1
dataloader = DataLoader(dataset, batch_size=batch_size, num_workers=args.num_workers)
dataloader_eval = DataLoader(dataset_eval, batch_size=batch_size, num_workers=args.num_workers)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = DGCL(
    num_features=dataset_num_features,
    hidden_dim=args.hidden_dim,
    num_layer=args.num_gc_layers,
    device=device,
    args=args
).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
model.eval()
emb, y = model.encoder.get_embeddings(dataloader_eval)
for epoch in range(1, epochs + 1):
    loss_all = 0
    model.train()
    for data in dataloader:
        data, data_aug = data
        optimizer.zero_grad()
        node_num, _ = data.x.size()
        data = data.to(device)
        x = model(data.x, data.edge_index, data.batch, data.num_graphs)
        edge_idx = data_aug.edge_index.numpy()
        _, edge_num = edge_idx.shape
        idx_not_missing = [n for n in range(node_num) if (n in edge_idx[0] or n in edge_idx[1])]
        node_num_aug = len(idx_not_missing)
        data_aug.x = data_aug.x[idx_not_missing]
        data_aug.batch = data.batch[idx_not_missing]
        idx_dict = {idx_not_missing[n]: n for n in range(node_num_aug)}
        edge_idx = [[idx_dict[edge_idx[0, n]], idx_dict[edge_idx[1, n]]] for n in range(edge_num) if
                    not edge_idx[0, n] == edge_idx[1, n]]
        data_aug.edge_index = torch.tensor(edge_idx).transpose_(0, 1)
        data_aug = data_aug.to(device)
        x_aug = model(data_aug.x, data_aug.edge_index, data_aug.batch, data_aug.num_graphs)
        loss = model.loss_cal(x, x_aug)
        loss_all += loss.item() * data.num_graphs
        loss.backward()
        optimizer.step()
    print('loss %.4f' % (loss_all / len(dataloader)))
    if log_interval > 0 and epoch % log_interval == 0:
        model.eval()
        emb, y = model.encoder.get_embeddings(dataloader_eval)
        result, result_val = evaluate_embedding(emb, y)
        accuracies['result'].append(result)
        accuracies['result_val'].append(result_val)

print(accuracies)

In [ ]:
args = arg_parse('HIV')
setup_seed(args.seed)

accuracies = {'result': [], 'result_val': []}
epochs = args.epoch
log_interval = args.log_interval
batch_size = args.batch
lr = args.lr
DS = args.DS
args.hidden_dim = int(args.hidden_dim // args.num_latent_factors) * args.num_latent_factors
path = '../data'
dataset = OGBDataset_aug(root=path, name="ogbg-molhiv", aug=args.aug).shuffle()
dataset_eval = OGBDataset_aug(root=path, name="ogbg-molhiv", aug='none').shuffle()
print(len(dataset))
print(dataset.get_num_feature())
try:
    dataset_num_features = dataset.get_num_feature()
except:
    dataset_num_features = 1
dataloader = DataLoader(dataset, batch_size=batch_size, num_workers=args.num_workers)
dataloader_eval = DataLoader(dataset_eval, batch_size=batch_size, num_workers=args.num_workers)
device = torch.device('cuda:7' if torch.cuda.is_available() else 'cpu')
model = DGCL(
    num_features=dataset_num_features,
    hidden_dim=args.hidden_dim,
    num_layer=args.num_gc_layers,
    device=device,
    args=args
).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
# model.eval()
# emb, y = model.encoder.get_embeddings(dataloader_eval)
for epoch in range(1, epochs + 1):
    loss_all = 0
    model.train()
    for data in dataloader:
        data, data_aug = data
        optimizer.zero_grad()
        node_num, _ = data.x.size()
        data.x = data.x.float()
        data = data.to(device)
        x = model(data.x, data.edge_index, data.batch, data.num_graphs)
        edge_idx = data_aug.edge_index.numpy()
        _, edge_num = edge_idx.shape
        idx_not_missing = [n for n in range(node_num) if (n in edge_idx[0] or n in edge_idx[1])]
        node_num_aug = len(idx_not_missing)
        data_aug.x = data_aug.x[idx_not_missing]
        data_aug.batch = data.batch[idx_not_missing]
        idx_dict = {idx_not_missing[n]: n for n in range(node_num_aug)}
        edge_idx = [[idx_dict[edge_idx[0, n]], idx_dict[edge_idx[1, n]]] for n in range(edge_num) if
                    not edge_idx[0, n] == edge_idx[1, n]]
        data_aug.edge_index = torch.tensor(edge_idx).transpose_(0, 1)
        data_aug.x = data_aug.x.float()
        data_aug = data_aug.to(device)
        x_aug = model(data_aug.x, data_aug.edge_index, data_aug.batch, data_aug.num_graphs)
        loss = model.loss_cal(x, x_aug)
        loss_all += loss.item() * data.num_graphs
        loss.backward()
        optimizer.step()
    print('loss %.4f' % (loss_all / len(dataloader)))
    if log_interval > 0 and epoch % log_interval == 0:
        model.eval()
        emb, y = model.encoder.get_embeddings(dataloader_eval)
        result, result_val = evaluate_embedding(emb, y)
        accuracies['result'].append(result)
        accuracies['result_val'].append(result_val)

with open("../result/acc", "a+") as f:
    f.write(accuracies + "\n")

print(accuracies)

In [ ]:
a = torch.LongTensor([1,2,3])

In [ ]:
a.to(torch.float)